In [1]:
import pandas as pd
import numpy as np
from PIL import Image
from PIL import ImageOps
import PIL
import torch, torchvision
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from dataset import *
import random
import math
import seaborn
from  matplotlib import pyplot
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable
import time
from torch.optim import lr_scheduler

In [2]:
input_size = 10       # The image size = 28 x 28 = 784
hidden_size = 100      # The number of nodes at the hidden layer
num_classes = 5       # The number of output classes. In this case, from 0 to 9
num_epochs = 10         # The number of times entire dataset is trained
batch_size = 256       # The size of input data took for one iteration
learning_rate = 0.001  # The speed of convergence
non_pos_ratio = 10
weight_decay=5e-4

In [4]:
train_transform = transforms.Compose([
        transforms.RandomResizedCrop(200, scale=(1, 1), ratio=(1, 1)),
        transforms.RandomRotation((-90,90)),
        torchvision.transforms.RandomVerticalFlip(p=0.5),
        torchvision.transforms.RandomHorizontalFlip(p=0.5),
#         torchvision.transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0, hue=0),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.3019],
                             std=[0.1909])
    ])
test_transform = transforms.Compose([
        transforms.RandomResizedCrop(200, scale=(1, 1), ratio=(1, 1)),
#         transforms.RandomRotation((-90,90)),
#         torchvision.transforms.RandomVerticalFlip(p=0.5),
#         torchvision.transforms.RandomHorizontalFlip(p=0.5),
#         torchvision.transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0, hue=0),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.3019],
                             std=[0.1909])
    ])

In [5]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()                    # Inherited from the parent class nn.Module
        self.fc1 = nn.Linear(input_size, hidden_size)  # 1st Full-Connected Layer: 10 (input data) -> 500 (hidden node)
        self.relu = nn.ReLU()                          # Non-Linear ReLU Layer: max(0,x)
        self.fc2 = nn.Linear(hidden_size, num_classes) # 2nd Full-Connected Layer: 500 (hidden node) -> 5 (output class)
#         self.relu = nn.ReLU()                          # Non-Linear ReLU Layer: max(0,x)
#         self.fc3 = nn.Linear(hidden_size, num_classes) # 3rd Full-Connected Layer: 500 (hidden node) -> 5 (output class)
    
    def forward(self, x):                              # Forward pass: stacking each layer together
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
#         out = self.relu(out)
#         out = self.fc3(out)
        return out

In [6]:
net = Net(input_size, hidden_size, num_classes)

In [7]:
use_gpu = torch.cuda.is_available()

if use_gpu:
    print("GPU in use")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

classes = ["pos","neg","pos_o","nuc","non"]
num_of_classes = len(classes)

model_uniform = torch.load('/home/rliu/defect_classifier/models/python/res34_600epo_uniform_01-07-18.model')
model_uniform.eval()
model_hard = torch.load('/home/rliu/defect_classifier/models/python/res34_600epo_hard_01-07-18.model')
model_hard.eval()


if use_gpu:
#     model_uniform = torch.nn.DataParallel(model_uniform)
    model_uniform.to(device)
#     model_hard = torch.nn.DataParallel(model_hard)
    model_hard.to(device)
    net.to(device)

GPU in use


In [9]:
weights = [1.0, 1.0, 1.0, 1.0, 1.0/non_pos_ratio]
class_weights = torch.FloatTensor(weights).to(device)
criterion = nn.CrossEntropyLoss(weight = class_weights)
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [12]:
since = time.time()
best_model_wts = net.state_dict()
best_acc = 0.0
for epoch in range(num_epochs):
    trainset = defectDataset_df(df = split_and_sample(method = 'yolo',n_samples = 1995, non_pos_ratio=non_pos_ratio), window_size = window_size,
                                             transforms=train_transform)
    trainloader = torch.utils.data.DataLoader(trainset,
                                                 batch_size=batch_size, shuffle=True,
                                                 num_workers=8, drop_last=True)
    print("trainloader ready!")

    testset = defectDataset_df(df = split_and_sample(df_labels = pd.read_csv('/home/rliu/yolo2/v2_pytorch_yolo2/data/an_data/VOCdevkit/VOC2007/csv_labels/test.csv', sep=" "), 
                                                     df_yolo = pd.read_csv('/home/rliu/github/defect_classifier/yolo2_dm/results/test_yolo.csv', sep=' '),
                                            method = 'yolo',n_samples = 800), window_size = window_size, transforms=test_transform)
    testloader = torch.utils.data.DataLoader(testset,
                                                 batch_size=batch_size, shuffle=True,
                                                 num_workers=8)
    print("testloader ready!")
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)
    scheduler.step()
    model_uniform.train(False)
    model_hard.train(False)
    net.train(True)
    running_loss = 0.0
    running_corrects = 0
    for data in trainloader:
        
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        with torch.no_grad():
            outputs_uniform = model_uniform(inputs)
            outputs_hard = model_hard(inputs)
        outputs_in = torch.cat((outputs_uniform, outputs_hard), dim=1)
        outputs_out = net(outputs_in)
        _, preds = torch.max(outputs_out.data, 1)
        loss = criterion(outputs_out, labels)

        optimizer.zero_grad()                             # Intialize the hidden weight to all zeros
        loss.backward()                                   # Backward pass: compute the weight
        optimizer.step()                                  # Optimizer: update the weights of hidden nodes

#         if (i+1) % 100 == 0:                              # Logging
#             print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
#                  %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.item()))

        
        # statistics
        iter_loss = loss.item()
        correct = torch.sum(preds == labels.data).item()
        batch_accuracy = correct / batch_size
        running_loss += loss.item()
        running_corrects_tensor = torch.sum(preds == labels.data)
        running_corrects += running_corrects_tensor.item()        
        epoch_loss = running_loss / len(trainset)
        epoch_acc = running_corrects / len(trainset)

        print('{} Loss: {:.4f} Acc: {:.4f} batch_loss: {:.4f} correct: {:d} batch_accuracy: {:.4f}'.format(
            "train", epoch_loss, epoch_acc, iter_loss, correct, batch_accuracy))
        
    correct = 0
    total = 0
    class_correct = list(0. for i in range(5))
    class_total = list(0. for i in range(5))    
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs_uniform = model_uniform(inputs)
            outputs_hard = model_hard(inputs)
            outputs_in = torch.cat((outputs_uniform, outputs_hard), dim=1)
            outputs_out = net(outputs_in)
            _, predicted = torch.max(outputs_out.data, 1)
            c = (predicted == labels).squeeze()
            for i in range(batch_size):
                if len(labels) == batch_size:
                    label = labels[i]
                    class_correct[label] += c[i].item()
                    class_total[label] += 1
                    correct += c[i].item()
                    total += 1
#             if len(labels) == batch_size:
#                 total += labels.size(0)
#                 correct += (predicted == labels).sum().item()
    #         print(predicted)
    #         print(labels)
    #       print('processed: %d' % total)
    #       print('correct: %d' % correct)
        print('Accuracy of the network on the test images: %.5f %%' % (100 * correct / total))
        for i in range(5):
            print('Accuracy of %5s : %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))
        print('class total: ',class_total)
        print('class correct: ',class_correct)
        print('total: ', total)
        print('correct: ', correct)
time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))
print('Best val Acc: {:4f}'.format(best_acc))

# load best model weights
net.load_state_dict(best_model_wts)

trainloader ready!
testloader ready!
Epoch 0/9
----------
train Loss: 0.0002 Acc: 0.0062 batch_loss: 4.3978 correct: 172 batch_accuracy: 0.6719
train Loss: 0.0003 Acc: 0.0111 batch_loss: 3.2392 correct: 138 batch_accuracy: 0.5391
train Loss: 0.0004 Acc: 0.0149 batch_loss: 3.2827 correct: 107 batch_accuracy: 0.4180
train Loss: 0.0005 Acc: 0.0182 batch_loss: 2.7586 correct: 91 batch_accuracy: 0.3555
train Loss: 0.0006 Acc: 0.0213 batch_loss: 2.7285 correct: 87 batch_accuracy: 0.3398
train Loss: 0.0007 Acc: 0.0248 batch_loss: 2.3072 correct: 99 batch_accuracy: 0.3867
train Loss: 0.0007 Acc: 0.0281 batch_loss: 2.1693 correct: 91 batch_accuracy: 0.3555
train Loss: 0.0008 Acc: 0.0322 batch_loss: 1.8290 correct: 114 batch_accuracy: 0.4453
train Loss: 0.0009 Acc: 0.0375 batch_loss: 1.6473 correct: 147 batch_accuracy: 0.5742
train Loss: 0.0009 Acc: 0.0419 batch_loss: 1.4680 correct: 124 batch_accuracy: 0.4844
train Loss: 0.0010 Acc: 0.0465 batch_loss: 1.3619 correct: 130 batch_accuracy: 0.5078


train Loss: 0.0024 Acc: 0.7307 batch_loss: 0.2291 correct: 237 batch_accuracy: 0.9258
train Loss: 0.0024 Acc: 0.7393 batch_loss: 0.2151 correct: 241 batch_accuracy: 0.9414
train Loss: 0.0024 Acc: 0.7481 batch_loss: 0.2452 correct: 245 batch_accuracy: 0.9570
train Loss: 0.0024 Acc: 0.7569 batch_loss: 0.2396 correct: 245 batch_accuracy: 0.9570
train Loss: 0.0024 Acc: 0.7656 batch_loss: 0.1902 correct: 245 batch_accuracy: 0.9570
train Loss: 0.0024 Acc: 0.7743 batch_loss: 0.2648 correct: 241 batch_accuracy: 0.9414
train Loss: 0.0024 Acc: 0.7830 batch_loss: 0.1645 correct: 244 batch_accuracy: 0.9531
train Loss: 0.0024 Acc: 0.7917 batch_loss: 0.2061 correct: 243 batch_accuracy: 0.9492
train Loss: 0.0024 Acc: 0.8004 batch_loss: 0.1739 correct: 244 batch_accuracy: 0.9531
train Loss: 0.0025 Acc: 0.8088 batch_loss: 0.2428 correct: 233 batch_accuracy: 0.9102
train Loss: 0.0025 Acc: 0.8176 batch_loss: 0.1732 correct: 246 batch_accuracy: 0.9609
train Loss: 0.0025 Acc: 0.8264 batch_loss: 0.1774 corr

train Loss: 0.0005 Acc: 0.6770 batch_loss: 0.1652 correct: 241 batch_accuracy: 0.9414
train Loss: 0.0005 Acc: 0.6859 batch_loss: 0.1082 correct: 248 batch_accuracy: 0.9688
train Loss: 0.0005 Acc: 0.6944 batch_loss: 0.1819 correct: 238 batch_accuracy: 0.9297
train Loss: 0.0005 Acc: 0.7029 batch_loss: 0.1547 correct: 237 batch_accuracy: 0.9258
train Loss: 0.0005 Acc: 0.7116 batch_loss: 0.1967 correct: 243 batch_accuracy: 0.9492
train Loss: 0.0005 Acc: 0.7203 batch_loss: 0.1367 correct: 244 batch_accuracy: 0.9531
train Loss: 0.0005 Acc: 0.7290 batch_loss: 0.1284 correct: 243 batch_accuracy: 0.9492
train Loss: 0.0005 Acc: 0.7376 batch_loss: 0.1646 correct: 239 batch_accuracy: 0.9336
train Loss: 0.0005 Acc: 0.7462 batch_loss: 0.1667 correct: 240 batch_accuracy: 0.9375
train Loss: 0.0005 Acc: 0.7547 batch_loss: 0.1528 correct: 238 batch_accuracy: 0.9297
train Loss: 0.0005 Acc: 0.7633 batch_loss: 0.1860 correct: 241 batch_accuracy: 0.9414
train Loss: 0.0005 Acc: 0.7721 batch_loss: 0.1431 corr

train Loss: 0.0003 Acc: 0.5211 batch_loss: 0.0848 correct: 248 batch_accuracy: 0.9688
train Loss: 0.0003 Acc: 0.5299 batch_loss: 0.0963 correct: 247 batch_accuracy: 0.9648
train Loss: 0.0003 Acc: 0.5388 batch_loss: 0.1166 correct: 247 batch_accuracy: 0.9648
train Loss: 0.0003 Acc: 0.5475 batch_loss: 0.0992 correct: 245 batch_accuracy: 0.9570
train Loss: 0.0003 Acc: 0.5564 batch_loss: 0.0856 correct: 246 batch_accuracy: 0.9609
train Loss: 0.0003 Acc: 0.5651 batch_loss: 0.1267 correct: 245 batch_accuracy: 0.9570
train Loss: 0.0003 Acc: 0.5738 batch_loss: 0.1169 correct: 242 batch_accuracy: 0.9453
train Loss: 0.0003 Acc: 0.5826 batch_loss: 0.1507 correct: 245 batch_accuracy: 0.9570
train Loss: 0.0003 Acc: 0.5911 batch_loss: 0.1109 correct: 239 batch_accuracy: 0.9336
train Loss: 0.0003 Acc: 0.5998 batch_loss: 0.1493 correct: 242 batch_accuracy: 0.9453
train Loss: 0.0003 Acc: 0.6086 batch_loss: 0.0967 correct: 247 batch_accuracy: 0.9648
train Loss: 0.0004 Acc: 0.6174 batch_loss: 0.1590 corr

train Loss: 0.0002 Acc: 0.3659 batch_loss: 0.1060 correct: 243 batch_accuracy: 0.9492
train Loss: 0.0002 Acc: 0.3748 batch_loss: 0.0950 correct: 248 batch_accuracy: 0.9688
train Loss: 0.0002 Acc: 0.3835 batch_loss: 0.1368 correct: 242 batch_accuracy: 0.9453
train Loss: 0.0002 Acc: 0.3922 batch_loss: 0.1377 correct: 244 batch_accuracy: 0.9531
train Loss: 0.0002 Acc: 0.4008 batch_loss: 0.1269 correct: 241 batch_accuracy: 0.9414
train Loss: 0.0002 Acc: 0.4097 batch_loss: 0.1046 correct: 248 batch_accuracy: 0.9688
train Loss: 0.0002 Acc: 0.4185 batch_loss: 0.0939 correct: 245 batch_accuracy: 0.9570
train Loss: 0.0002 Acc: 0.4273 batch_loss: 0.0792 correct: 247 batch_accuracy: 0.9648
train Loss: 0.0002 Acc: 0.4362 batch_loss: 0.1047 correct: 248 batch_accuracy: 0.9688
train Loss: 0.0002 Acc: 0.4450 batch_loss: 0.1339 correct: 247 batch_accuracy: 0.9648
train Loss: 0.0002 Acc: 0.4537 batch_loss: 0.0999 correct: 241 batch_accuracy: 0.9414
train Loss: 0.0002 Acc: 0.4623 batch_loss: 0.1645 corr

...

train Loss: 0.0004 Acc: 0.8923 batch_loss: 0.0736 correct: 246 batch_accuracy: 0.9609
train Loss: 0.0004 Acc: 0.9011 batch_loss: 0.1388 correct: 245 batch_accuracy: 0.9570
train Loss: 0.0004 Acc: 0.9099 batch_loss: 0.0790 correct: 245 batch_accuracy: 0.9570
train Loss: 0.0004 Acc: 0.9186 batch_loss: 0.1110 correct: 244 batch_accuracy: 0.9531
train Loss: 0.0004 Acc: 0.9271 batch_loss: 0.1790 correct: 237 batch_accuracy: 0.9258
train Loss: 0.0004 Acc: 0.9358 batch_loss: 0.0769 correct: 244 batch_accuracy: 0.9531
train Loss: 0.0004 Acc: 0.9448 batch_loss: 0.0514 correct: 251 batch_accuracy: 0.9805
train Loss: 0.0004 Acc: 0.9534 batch_loss: 0.1267 correct: 240 batch_accuracy: 0.9375
Accuracy of the network on the test images: 84.73958 %
Accuracy of   pos : 87 %
Accuracy of   neg : 89 %
Accuracy of pos_o : 78 %
Accuracy of   nuc : 72 %
Accuracy of   non : 96 %
class total:  [767.0, 769.0, 766.0, 767.0, 771.0]
class correct:  [669.0, 689.0, 599.0, 556.0, 741.0]
total:  3840
correct:  3254
tr

train Loss: 0.0003 Acc: 0.7340 batch_loss: 0.1168 correct: 242 batch_accuracy: 0.9453
train Loss: 0.0003 Acc: 0.7426 batch_loss: 0.1394 correct: 239 batch_accuracy: 0.9336
train Loss: 0.0004 Acc: 0.7515 batch_loss: 0.1215 correct: 247 batch_accuracy: 0.9648
train Loss: 0.0004 Acc: 0.7601 batch_loss: 0.1340 correct: 242 batch_accuracy: 0.9453
train Loss: 0.0004 Acc: 0.7689 batch_loss: 0.0635 correct: 244 batch_accuracy: 0.9531
train Loss: 0.0004 Acc: 0.7774 batch_loss: 0.0947 correct: 240 batch_accuracy: 0.9375
train Loss: 0.0004 Acc: 0.7863 batch_loss: 0.0968 correct: 248 batch_accuracy: 0.9688
train Loss: 0.0004 Acc: 0.7952 batch_loss: 0.2014 correct: 248 batch_accuracy: 0.9688
train Loss: 0.0004 Acc: 0.8037 batch_loss: 0.1669 correct: 238 batch_accuracy: 0.9297
train Loss: 0.0004 Acc: 0.8126 batch_loss: 0.0784 correct: 247 batch_accuracy: 0.9648
train Loss: 0.0004 Acc: 0.8214 batch_loss: 0.0757 correct: 248 batch_accuracy: 0.9688
train Loss: 0.0004 Acc: 0.8301 batch_loss: 0.1955 corr

train Loss: 0.0003 Acc: 0.5769 batch_loss: 0.0856 correct: 248 batch_accuracy: 0.9688
train Loss: 0.0003 Acc: 0.5857 batch_loss: 0.0947 correct: 244 batch_accuracy: 0.9531
train Loss: 0.0003 Acc: 0.5944 batch_loss: 0.0881 correct: 244 batch_accuracy: 0.9531
train Loss: 0.0003 Acc: 0.6032 batch_loss: 0.0978 correct: 246 batch_accuracy: 0.9609
train Loss: 0.0003 Acc: 0.6121 batch_loss: 0.0691 correct: 249 batch_accuracy: 0.9727
train Loss: 0.0003 Acc: 0.6208 batch_loss: 0.1362 correct: 241 batch_accuracy: 0.9414
train Loss: 0.0003 Acc: 0.6294 batch_loss: 0.0770 correct: 242 batch_accuracy: 0.9453
train Loss: 0.0003 Acc: 0.6381 batch_loss: 0.1396 correct: 242 batch_accuracy: 0.9453
train Loss: 0.0003 Acc: 0.6469 batch_loss: 0.1262 correct: 246 batch_accuracy: 0.9609
train Loss: 0.0003 Acc: 0.6558 batch_loss: 0.1028 correct: 249 batch_accuracy: 0.9727
train Loss: 0.0003 Acc: 0.6646 batch_loss: 0.0865 correct: 244 batch_accuracy: 0.9531
train Loss: 0.0003 Acc: 0.6731 batch_loss: 0.1623 corr

train Loss: 0.0002 Acc: 0.4187 batch_loss: 0.1066 correct: 244 batch_accuracy: 0.9531
train Loss: 0.0002 Acc: 0.4272 batch_loss: 0.1561 correct: 236 batch_accuracy: 0.9219
train Loss: 0.0002 Acc: 0.4358 batch_loss: 0.1939 correct: 240 batch_accuracy: 0.9375
train Loss: 0.0002 Acc: 0.4445 batch_loss: 0.1068 correct: 245 batch_accuracy: 0.9570
train Loss: 0.0002 Acc: 0.4532 batch_loss: 0.1043 correct: 242 batch_accuracy: 0.9453
train Loss: 0.0002 Acc: 0.4618 batch_loss: 0.1302 correct: 240 batch_accuracy: 0.9375
train Loss: 0.0002 Acc: 0.4705 batch_loss: 0.0915 correct: 243 batch_accuracy: 0.9492
train Loss: 0.0002 Acc: 0.4792 batch_loss: 0.0878 correct: 244 batch_accuracy: 0.9531
train Loss: 0.0002 Acc: 0.4880 batch_loss: 0.1667 correct: 246 batch_accuracy: 0.9609
train Loss: 0.0002 Acc: 0.4968 batch_loss: 0.1702 correct: 245 batch_accuracy: 0.9570
train Loss: 0.0002 Acc: 0.5056 batch_loss: 0.1159 correct: 245 batch_accuracy: 0.9570
train Loss: 0.0003 Acc: 0.5142 batch_loss: 0.0975 corr

train Loss: 0.0001 Acc: 0.2625 batch_loss: 0.1123 correct: 246 batch_accuracy: 0.9609
train Loss: 0.0001 Acc: 0.2712 batch_loss: 0.1159 correct: 241 batch_accuracy: 0.9414
train Loss: 0.0001 Acc: 0.2800 batch_loss: 0.1251 correct: 246 batch_accuracy: 0.9609
train Loss: 0.0001 Acc: 0.2888 batch_loss: 0.1014 correct: 245 batch_accuracy: 0.9570
train Loss: 0.0001 Acc: 0.2974 batch_loss: 0.1735 correct: 242 batch_accuracy: 0.9453
train Loss: 0.0001 Acc: 0.3063 batch_loss: 0.0833 correct: 249 batch_accuracy: 0.9727
train Loss: 0.0001 Acc: 0.3150 batch_loss: 0.1492 correct: 241 batch_accuracy: 0.9414
train Loss: 0.0002 Acc: 0.3238 batch_loss: 0.0813 correct: 246 batch_accuracy: 0.9609
train Loss: 0.0002 Acc: 0.3323 batch_loss: 0.1102 correct: 237 batch_accuracy: 0.9258
train Loss: 0.0002 Acc: 0.3410 batch_loss: 0.1396 correct: 244 batch_accuracy: 0.9531
train Loss: 0.0002 Acc: 0.3496 batch_loss: 0.1088 correct: 240 batch_accuracy: 0.9375
train Loss: 0.0002 Acc: 0.3585 batch_loss: 0.1310 corr

In [ ]:
preds

In [81]:
print(inputs.shape)
print(outputs_uniform.shape)
print(outputs_in.shape)
print(outputs_out.shape)

torch.Size([256, 1, 200, 200])
torch.Size([256, 5])
torch.Size([256, 10])
torch.Size([256, 5])


In [62]:
criterion = nn.CrossEntropyLoss()

In [63]:
use_gpu = torch.cuda.is_available()

if use_gpu:
    print("GPU in use")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

classes = ["pos","neg","pos_o","nuc","non"]
num_of_classes = len(classes)

model_uniform = torch.load('/home/rliu/defect_classifier/models/python/res34_600epo_uniform_01-07-18.model')
model_uniform.eval()
model_hard = torch.load('/home/rliu/defect_classifier/models/python/res34_600epo_hard_01-07-18.model')
model_hard.eval()
if use_gpu:
#     model_uniform = torch.nn.DataParallel(model_uniform)
    model_uniform.to(device)
#     model_hard = torch.nn.DataParallel(model_hard)
    model_hard.to(device)
    
    
batch_size = 5
trainset = defectDataset_df(df = split_and_sample(method = 'yolo',n_samples = 1995, non_pos_ratio = non_pos_ratio), window_size = window_size, transforms=data_transform)
trainloader = torch.utils.data.DataLoader(trainset,
                                     batch_size=batch_size, shuffle=True,
                                     num_workers=4, drop_last=True)
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
images, labels = images.to(device), labels.to(device)
outputs_uniform = model_uniform(images)
outputs_hard = model_hard(images)
outputs = torch.cat((outputs_uniform, outputs_hard), dim=1)
_, preds = torch.max(outputs.data, 1)
loss = criterion(outputs, labels)

GPU in use


In [64]:
print(outputs_uniform)
print(outputs_hard)
print(loss)

tensor([[-12.3108,   3.8134, -12.1610,  -8.9865, -18.6308],
        [  4.1426, -12.6464, -16.8836, -13.2110, -13.5861],
        [ -9.9046, -12.1545,  -9.0868,   2.3950, -12.2668],
        [ -5.9917,   3.7852,  -9.3418, -10.6862, -11.3693],
        [  4.1033,  -7.9563,  -6.8923, -13.0635,  -7.5087]], device='cuda:0',
       grad_fn=<GatherBackward>)
tensor([[-10.9020,   2.6503, -13.7712, -12.7975, -13.3193],
        [  3.1850, -11.8032, -13.6199, -11.8008, -12.5677],
        [-13.4012, -12.1915, -14.3676,   1.9575, -15.2399],
        [ -7.2337,   2.6256, -18.3268, -10.4256,  -9.0024],
        [  3.2153, -10.9220, -13.6908, -12.3731, -11.7074]], device='cuda:0',
       grad_fn=<GatherBackward>)
tensor(0.3425, device='cuda:0', grad_fn=<NllLossBackward>)


In [54]:
labels

tensor([3, 0, 0, 3, 1])

In [3]:
train_dataset = datasets.MNIST(root='./data',
                           train=True,
                           transform=transforms.ToTensor(),
                           download=True)

test_dataset = datasets.MNIST(root='./data',
                           train=False,
                           transform=transforms.ToTensor())

In [51]:
torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

device(type='cuda', index=0)

In [4]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [5]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()                    # Inherited from the parent class nn.Module
        self.fc1 = nn.Linear(input_size, hidden_size)  # 1st Full-Connected Layer: 784 (input data) -> 500 (hidden node)
        self.relu = nn.ReLU()                          # Non-Linear ReLU Layer: max(0,x)
        self.fc2 = nn.Linear(hidden_size, num_classes) # 2nd Full-Connected Layer: 500 (hidden node) -> 10 (output class)
    
    def forward(self, x):                              # Forward pass: stacking each layer together
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [6]:
net = Net(input_size, hidden_size, num_classes)

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)

Net(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [16]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):   # Load a batch of images with its (index, data, class)
        images = Variable(images.view(-1, 28*28))         # Convert torch tensor to Variable: change image from a vector of size 784 to a matrix of 28 x 28
        labels = Variable(labels)
        
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()                             # Intialize the hidden weight to all zeros
        outputs = net(images)                             # Forward pass: compute the output class given a image
        loss = criterion(outputs, labels)                 # Compute the loss: difference between the output class and the pre-given label
        loss.backward()                                   # Backward pass: compute the weight
        optimizer.step()                                  # Optimizer: update the weights of hidden nodes
        
        if (i+1) % 100 == 0:                              # Logging
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.2542
Epoch [1/5], Step [200/600], Loss: 0.1384
Epoch [1/5], Step [300/600], Loss: 0.1731
Epoch [1/5], Step [400/600], Loss: 0.1142
Epoch [1/5], Step [500/600], Loss: 0.0950
Epoch [1/5], Step [600/600], Loss: 0.1930
Epoch [2/5], Step [100/600], Loss: 0.1513
Epoch [2/5], Step [200/600], Loss: 0.1833
Epoch [2/5], Step [300/600], Loss: 0.1233
Epoch [2/5], Step [400/600], Loss: 0.1183
Epoch [2/5], Step [500/600], Loss: 0.0871
Epoch [2/5], Step [600/600], Loss: 0.0677
Epoch [3/5], Step [100/600], Loss: 0.0196
Epoch [3/5], Step [200/600], Loss: 0.0825
Epoch [3/5], Step [300/600], Loss: 0.0880
Epoch [3/5], Step [400/600], Loss: 0.0757
Epoch [3/5], Step [500/600], Loss: 0.0220
Epoch [3/5], Step [600/600], Loss: 0.0282
Epoch [4/5], Step [100/600], Loss: 0.0166
Epoch [4/5], Step [200/600], Loss: 0.0539
Epoch [4/5], Step [300/600], Loss: 0.0231
Epoch [4/5], Step [400/600], Loss: 0.0305
Epoch [4/5], Step [500/600], Loss: 0.0996
Epoch [4/5], Step [600/600], Loss:

In [15]:
loss.item()

0.22127796709537506

In [ ]:
    use_gpu = torch.cuda.is_available()

    if use_gpu:
        print("GPU in use")

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    classes = ["pos","neg","pos_o","nuc","non"]
    num_of_classes = len(classes)

    model_uniform = torch.load('/home/rliu/defect_classifier/models/python/res34_600epo_uniform_01-07-18.model')
    model_uniform.eval()
    model_hard = torch.load('/home/rliu/defect_classifier/models/python/res34_600epo_uniform_01-07-18.model')
    model_hard.eval()
    if use_gpu:
        model_uniform = torch.nn.DataParallel(uniform)
        model_uniform.to(device)
        model_uniform = torch.nn.DataParallel(uniform)
        model_uniform.to(device)

In [18]:
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28))
    images, labels = images.to(device), labels.to(device)
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)  # Choose the best class from the output: The class with the best score
    total += labels.size(0)                    # Increment the total count
    correct += (predicted == labels).sum()     # Increment the correct count
    
print('Accuracy of the network on the 10K test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10K test images: 97 %
